In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Todas las librerias necesarias excepto la primera que es el algoritmo de clasificacion
from sklearn.neighbors import KNeighborsClassifier
from google.colab.patches import cv2_imshow
from google.colab import files
from skimage import exposure
from skimage import feature
import argparse #Parsear desde la linea de comandos
from imutils import paths
import imutils
import cv2
import glob
import re

In [0]:
all_images_train = [] #Contiene las imagenes sin el HOG
data = [] #Datos HOG de cada imagen
labels = [] #Datos de etiquetas que identifican cada imagen

images_train = glob.glob('/content/drive/My Drive/Colab Notebooks/cancer_images/mama_cancer_png/train' + '/*.png')

for i in images_train:
  if i.endswith("png"):
    all_images_train.append(i)

In [0]:
def splitfilename(filename):
    sname=""
    sext=""
    i=filename.rfind(".")
    if(i!=0):
        n=len(filename)
        j=n-i-1
        sname=filename[0:i]
        sext=filename[-j:]    
    return sname, sext

def split_final_name(filename):
  

In [0]:
for img in all_images_train:
  image = cv2.imread(img)
  name = splitfilename(img)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  edged = imutils.auto_canny(gray)
  H = feature.hog(gray, orientations=9, pixels_per_cell=(10, 10), cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1")
  data.append(H)
  labels.append(name)

In [0]:
#Mostrar datos del HOG
for i in labels:
  print(i)

for i in data:
  print(i)

In [0]:
#Entrenamiento por medio de KKN

print("Training KNN clasiffier")
model = KNeighborsClassifier(n_neighbors=3)
print(model.fit(data, labels))

Training KNN clasiffier
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')


In [0]:
all_images_test = []
images_test = glob.glob('/content/drive/My Drive/Colab Notebooks/cancer_images/mama_cancer_png/test' + '/*.png')

for i in images_test:
  if i.endswith("png"):
    all_images_test.append(i)

In [0]:
for img in all_images_test:
  image = cv2.imread(img)
  #name = splitfilename(img)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  (h, hog_image) = feature.hog(gray, orientations=9, pixels_per_cell=(10, 10), cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1", visualise=True)
  pred = model.predict(h.reshape(1, -1))[0]

  print(pred)
  print(type(pred))

  #Visualize HOG image
  hog_image = exposure.rescale_intensity(hog_image, out_range=(0, 255))
  hog_image = hog_image.astype("uint8")
  cv2_imshow(hog_image)

  text = str(img)
  #Show prediction
  cv2.putText(image, img, (5, 25), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (0, 200, 0), 3)
  cv2_imshow(image)
  cv2.waitKey(0)

  #cv2.putText(image, pred.putText(img, "Prediction"), (10,35), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1.0, (0, 255, 0), 3)
  #cv2_imshow("Test image #{}".format(img + 1), image)
  #cv2.waitKey(0)

AttributeError: ignored